-sandbox
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px; height: 163px">
</div>

# Project: Exploratory Data Analysis
Perform exploratory data analysis (EDA) to gain insights from a data lake.

## Audience
* Primary Audience: Data Analysts
* Additional Audiences: Data Engineers and Data Scientists

## Prerequisites
* Web browser: Chrome or Firefox
* Lesson: <a href="$./04-Querying-Files">Querying Files with SQL</a>
* Lesson: <a href="$./05-Querying-Lakes">Querying Data Lakes with SQL</a>
* Concept: <a href="https://www.w3schools.com/sql/" target="_blank">Basic SQL</a>

## Instructions

In `dbfs:/mnt/training/crime-data-2016`, there are a number of Parquet files containing 2016 crime data from seven United States cities:

* New York
* Los Angeles
* Chicago
* Philadelphia
* Dallas
* Boston


The data is cleaned up a little but has not been normalized. Each city reports crime data slightly differently, so you have to
examine the data for each city to determine how to query it properly.

Your job is to use some of this data to gain insights about certain kinds of crimes.

### Getting Started

Run the following cell to configure our "classroom."

In [4]:
%run ./Includes/Classroom-Setup

-sandbox
## Step 1

Start by creating temporary views for Los Angeles, Philadelphia, and Dallas

Use `CREATE TEMPORARY VIEW` to create named views for the files you choose. Use a similar syntax as `CREATE TABLE`:

```
CREATE OR REPLACE TEMPORARY VIEW name
  USING parquet
  OPTIONS (
    ...
  )
```

Use the following view names:

| City          | Table Name              | Path to DBFS file
| ------------- | ----------------------- | -----------------
| Los Angeles   | `CrimeDataLosAngeles`   | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet`
| Philadelphia  | `CrimeDataPhiladelphia` | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet`
| Dallas        | `CrimeDataDallas`       | `dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet`



<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You learned how to create a table from an external file in [Lesson 3]($./03-Accessing-Data). The syntax is exactly the same, except that you use `CREATE OR REPLACE TEMPORARY VIEW` instead of `CREATE TABLE IF EXISTS`.

#### Los Angeles

In [7]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CrimeDataLosAngeles 
USING parquet
OPTIONS(
path "dbfs:/mnt/training/crime-data-2016/Crime-Data-Los-Angeles-2016.parquet"
);


In [8]:
%python
# TEST - Run this cell to test your solution.

rowsLosAngeles = spark.sql('SELECT count(*) FROM CrimeDataLosAngeles').collect()[0][0]
dbTest("SQL-L7-crimeDataLA-count", 217945, rowsLosAngeles)

print("Tests passed!")

Tests passed!

#### Philadelphia

In [10]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CrimeDataPhiladelphia 
USING parquet
OPTIONS(
path "dbfs:/mnt/training/crime-data-2016/Crime-Data-Philadelphia-2016.parquet"
);

In [11]:
%python
# TEST - Run this cell to test your solution.

rowsPhiladelphia = spark.sql('SELECT count(*) FROM CrimeDataPhiladelphia').collect()[0][0]
dbTest("SQL-L7-crimeDataPA-count", 168664, rowsPhiladelphia)

print("Tests passed!")

Tests passed!

#### Dallas

In [13]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CrimeDataDallas 
USING parquet
OPTIONS(
path "dbfs:/mnt/training/crime-data-2016/Crime-Data-Dallas-2016.parquet"
);


In [14]:
%python
# TEST - Run this cell to test your solution.

rowsDallas = spark.sql('SELECT count(*) FROM CrimeDataDallas').collect()[0][0]
dbTest("SQL-L7-crimeDataDAL-count", 99642, rowsDallas) 

print("Tests passed!")

Tests passed!

-sandbox
## Step 2

For each table, examine the data to figure out how to extract _robbery_ statistics.

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> Each city uses different values to indicate robbery. Some cities use "larceny", "burglary", and "robbery".  These challenges are common in data lakes.  To simplify things, restrict yourself to only the word "robbery" (and not attempted-roberty, larceny, or burglary).

Explore the data for the three cities until you understand how each city records robbery information. If you don't want to worry about upper- or lower-case, remember that SQL has a `LOWER()` function that converts a column's value to lowercase.

Create a temporary view containing only the robbery-related rows, as shown in the table below.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** For each table, focus your efforts on the column listed below.

Focus on the following columns for each table:

| Table Name              | Robbery View Name     | Column
| ----------------------- | ----------------------- | -------------------------------
| `CrimeDataLosAngeles`   | `RobberyLosAngeles`   | `crimeCodeDescription`
| `CrimeDataPhiladelphia` | `RobberyPhiladelphia` | `ucr_general_description`
| `CrimeDataDallas`       | `RobberyDallas`       | `typeOfIncident`

#### Los Angeles

In [17]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberyLosAngeles AS
SELECT month(timeOccurred) AS month, crimeCodeDescription FROM CrimeDataLosAngeles 
WHERE lower(crimeCodeDescription) LIKE 'robbery'

In [18]:
%python
# TEST - Run this cell to test your solution.

totalLosAngeles = spark.sql("SELECT count(*) AS total FROM RobberyLosAngeles").collect()[0].total
dbTest("SQL-L7-robberyDataLA-count", 9048, totalLosAngeles)

print("Tests passed!")

Tests passed!

#### Philadelphia

In [20]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberyPhiladelphia AS
SELECT month(dispatch_date_time) AS month, ucr_general_description FROM CrimeDataPhiladelphia 
WHERE lower(ucr_general_description) LIKE 'robbery'

In [21]:
%python
# TEST - Run this cell to test your solution.

totalPhiladelphia = spark.sql("SELECT count(*) AS total FROM RobberyPhiladelphia").collect()[0].total
dbTest("SQL-L7-robberyDataPA-count", 6149, totalPhiladelphia)

print("Tests passed!")

Tests passed!

#### Dallas

In [23]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberyDallas AS
SELECT month(startingDateTime) AS month, typeOfIncident FROM CrimeDataDallas 
WHERE lower(typeOfIncident) LIKE 'robbery%'

In [24]:
%python
# TEST - Run this cell to test your solution.

totalDallas = spark.sql("SELECT count(*) AS total FROM RobberyDallas").collect()[0].total
dbTest("SQL-L7-robberyDataDAL-count", 6824, totalDallas)

print("Tests passed!")

Tests passed!

-sandbox
## Step 3

Now that you have views of only the robberies in each city, create temporary views for each city, summarizing the number of robberies in each month.

Your views must contain two columns:
* `month`: The month number (e.g., 1 for January, 2 for February, etc.)
* `robberies`: The total number of robberies in the month

Use the following temporary view names and date columns:


| City          | View Name     | Date Column 
| ------------- | ------------- | -------------
| Los Angeles   | `RobberiesByMonthLosAngeles` | `timeOccurred`
| Philadelphia  | `RobberiesByMonthPhiladelphia` | `dispatch_date_time`
| Dallas        | `RobberiesByMonthDallas` | `startingDateTime`

<img alt="Side Note" title="Side Note" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.05em; transform:rotate(15deg)" src="https://files.training.databricks.com/static/images/icon-note.webp"/> For each city, figure out which column contains the date of the incident. Then, extract the month from that date.

#### Los Angeles

In [27]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberiesByMonthLosAngeles AS
SELECT month, count(1) AS robberies FROM  RobberyLosAngeles 
GROUP BY month
ORDER BY month;

In [28]:
%python
# TEST - Run this cell to test your solution.

la = { r.month : r.robberies for r in spark.sql("SELECT * FROM RobberiesByMonthLosAngeles").collect() }
dbTest("SQL-L7-robberyByMonthLA-counts", {1: 719, 2: 675, 3: 709, 4: 713, 5: 790, 6: 698, 7: 826, 8: 765, 9: 722, 10: 814, 11: 764, 12: 853}, la)

print("Tests passed!")

Tests passed!

#### Philadelphia

In [30]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberiesByMonthPhiladelphia AS
SELECT month, count(1) AS robberies FROM  RobberyPhiladelphia 
GROUP BY month
ORDER BY month;

In [31]:
%python
# TEST - Run this cell to test your solution.

philadelphia = { r.month : r.robberies for r in spark.sql("SELECT * FROM RobberiesByMonthPhiladelphia").collect() }
dbTest("SQL-L7-robberyByMonthPA-counts", {1: 520, 2: 416, 3: 432, 4: 466, 5: 533, 6: 509, 7: 537, 8: 561, 9: 514, 10: 572, 11: 545, 12: 544}, philadelphia)

print("Tests passed!")

Tests passed!

#### Dallas

In [33]:
%sql
CREATE OR REPLACE TEMPORARY VIEW RobberiesByMonthDallas AS
SELECT month, count(1) AS robberies FROM  RobberyDallas 
GROUP BY month
ORDER BY month;

In [34]:
%python
# TEST - Run this cell to test your solution.

dallas = { r.month : r.robberies for r in spark.sql("SELECT * FROM RobberiesByMonthDallas").collect() }
dbTest("SQL-L7-robberyByMonthDAL-counts", {1: 743, 2: 435, 3: 412, 4: 594, 5: 615, 6: 495, 7: 535, 8: 627, 9: 512, 10: 603, 11: 589, 12: 664}, dallas)

print("Tests passed!")

Tests passed!

-sandbox

## Step 4

Plot the robberies per month for each of your three cities, producing a plot similar to the following:

<img src="https://files.training.databricks.com/images/eLearning/robberies-by-month.png" style="max-width: 700px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

When you first run your cell, you'll get an HTML table as the result. To configure the plot,

1. Click the graph button
2. If the plot doesn't look correct, click the **Plot Options** button
3. Configure the plot similar to the following example

<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-1.png" style="width: 440px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>
<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-2.png" style="width: 268px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>
<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-3.png" style="width: 362px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

#### Los Angeles

In [37]:
%sql
SELECT * FROM RobberiesByMonthLosAngeles;

month,robberies
1,719
2,675
3,709
4,713
5,790
6,698
7,826
8,765
9,722
10,814


#### Philadelphia

In [39]:
%sql
SELECT * FROM RobberiesByMonthPhiladelphia;

month,robberies
1,520
2,416
3,432
4,466
5,533
6,509
7,537
8,561
9,514
10,572


#### Dallas

In [41]:
%sql
SELECT * FROM RobberiesByMonthDallas;

month,robberies
1,743
2,435
3,412
4,594
5,615
6,495
7,535
8,627
9,512
10,603


-sandbox
## Step 5

Create another temporary view called `CombinedRobberiesByMonth`, that combines all three robberies-per-month views into one.
In creating this view, add a new column called `city`, that identifies the city associated with each row.
The final view will have the following columns:

* `city`: The name of the city associated with the row (Use the strings "Los Angeles", "Philadelphia", and "Dallas".)
* `month`: The month number associated with the row
* `robbery`: The number of robbery in that month (for that city)

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** You may want to use `UNION` in this example to combine the three datasets.

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** In Databricks, all table schemas are immutable and therefore standard SQL commands such as `ALTER…ADD` and `UPDATE…SET` do not work for adding the new "city" column. 

Instead, new columns can be added by simply naming them in the `SELECT` statement within the `CREATE OR REPLACE TEMPORARY VIEW` statement.

In [43]:
%sql
CREATE OR REPLACE TEMPORARY VIEW CombinedRobberiesByMonth AS
SELECT "Los Angeles" AS city, month, robberies FROM RobberiesByMonthLosAngeles
UNION 
SELECT "Philadelphia" AS city, month, robberies FROM RobberiesByMonthPhiladelphia
UNION 
SELECT "Dallas" AS city, month, robberies FROM RobberiesByMonthDallas
ORDER BY city, month;

In [44]:
%python
# TEST - Run this cell to test your solution.

results = [ (r.city, r.month, r.robberies) for r in spark.sql("SELECT * FROM CombinedRobberiesByMonth").collect() ]

dbTest("SQL-L7-combinedRobberiesByMonth-counts-0", (u'Dallas', 1, 743), results[0])
dbTest("SQL-L7-combinedRobberiesByMonth-counts-10", (u'Dallas', 11, 589) , results[10])
dbTest("SQL-L7-combinedRobberiesByMonth-counts-20", (u'Los Angeles', 9, 722), results[20])

print("Tests passed!")

Tests passed!

-sandbox
## Step 6

Graph the contents of `CombinedRobberiesByMonth`, producing a graph similar to the following. (The diagram below deliberately
uses different data.)

<img src="https://files.training.databricks.com/images/eLearning/combined-homicides.png" style="width: 800px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

Adjust the plot options to configure the plot properly, as shown below:

<img src="https://files.training.databricks.com/images/eLearning/capstone-plot-4.png" style="width: 362px; margin: 10px; border: 1px solid #aaaaaa; border-radius: 10px 10px 10px 10px"/>

<img alt="Hint" title="Hint" style="vertical-align: text-bottom; position: relative; height:1.75em; top:0.3em" src="https://files.training.databricks.com/static/images/icon-light-bulb.svg"/>&nbsp;**Hint:** Order your results by `month`, then `city`.

In [46]:
%sql
SELECT * FROM CombinedRobberiesByMonth;

city,month,robberies
Dallas,1,743
Dallas,2,435
Dallas,3,412
Dallas,4,594
Dallas,5,615
Dallas,6,495
Dallas,7,535
Dallas,8,627
Dallas,9,512
Dallas,10,603


## Step 7

While the above graph is interesting, it's flawed: it's comparing the raw numbers of robberies, not the per capita robbery rates.

The table (already created) called `CityData`  contains, among other data, estimated 2016 population values for all United States cities
with populations of at least 100,000. (The data is from [Wikipedia](https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population).)

* Use the population values in that table to normalize the robberies so they represent per-capita values (i.e. total robberies divided by population)
* Save your results in a temporary view called `RobberyRatesByCity`
* The robbery rate value must be stored in a new column, `robberyRate`

Next, graph the results, as above.

In [48]:
%sql
select * from RobberyRatesByCity;

city,month,robberyRate
Dallas,1,5.540331450515445E-4
Dallas,2,3.243666461607293E-4
Dallas,3,3.0721622578901256E-4
Dallas,4,4.429282478608579E-4
Dallas,5,4.585873273306862E-4
Dallas,6,3.6910687321738155E-4
Dallas,7,3.9893369125514977E-4
Dallas,8,4.6753537274201666E-4
Dallas,9,3.8178327088343305E-4
Dallas,10,4.4963928191935575E-4


In [49]:
%python
# TEST - Run this cell to test your solution.

results = [ (r.city, r.month, "{0:0.6f}".format(r.robberyRate)) for r in spark.sql("SELECT * FROM RobberyRatesByCity").collect() ]

dbTest("SQL-L7-roberryRatesByCity-counts-0", (u'Dallas', 1, '0.000554'), results[0])
dbTest("SQL-L7-roberryRatesByCity-counts-10", (u'Los Angeles', 11, '0.000192'), results[10])
dbTest("SQL-L7-roberryRatesByCity-counts-20", (u'Philadelphia', 7, '0.000343'), results[20])

print("Tests passed!")

--------------------------------------------------------------------------- 
 AssertionError Traceback (most recent call last)
 <command-1218674622327223> in <module> () 
 4 
 5 dbTest ( "SQL-L7-roberryRatesByCity-counts-0" , ( u'Dallas' , 1 , '0.000554' ) , results [ 0 ] ) 
 ----> 6 dbTest ( "SQL-L7-roberryRatesByCity-counts-10" , ( u'Los Angeles' , 11 , '0.000192' ) , results [ 10 ] ) 
 7 dbTest ( "SQL-L7-roberryRatesByCity-counts-20" , ( u'Philadelphia' , 7 , '0.000343' ) , results [ 20 ] ) 
 8 

 <command-1218674622327229> in dbTest (id, expected, result) 
 1 def dbTest ( id , expected , result ) : 
 ----> 2 assert str ( expected ) == str ( result ) , "{} does not equal expected {}" . format ( result , expected ) 

 AssertionError : (u'Dallas', 11, '0.000439') does not equal expected (u'Los Angeles', 11, '0.000192')

## IMPORTANT Next Steps
* Please complete this short [feedback survey](https://www.surveymonkey.com/r/WBMS7CG).  Your input is extremely important and will shape future development.
* Congratulations, you have completed the Spark SQL course!

## References

The crime data used in this notebook comes from the following locations:

| City          | Original Data 
| ------------- | -------------
| Boston        | <a href="https://data.boston.gov/group/public-safety" target="_blank">https&#58;//data.boston.gov/group/public-safety</a>
| Chicago       | <a href="https://data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2" target="_blank">https&#58;//data.cityofchicago.org/Public-Safety/Crimes-2001-to-present/ijzp-q8t2</a>
| Dallas        | <a href="https://www.dallasopendata.com/Public-Safety/Police-Incidents/tbnj-w5hb/data" target="_blank">https&#58;//www.dallasopendata.com/Public-Safety/Police-Incidents/tbnj-w5hb/data</a>
| Los Angeles   | <a href="https://data.lacity.org/A-Safe-City/Crime-Data-From-2010-to-Present/y8tr-7khq" target="_blank">https&#58;//data.lacity.org/A-Safe-City/Crime-Data-From-2010-to-Present/y8tr-7khq</a>
| New Orleans   | <a href="https://data.nola.gov/Public-Safety-and-Preparedness/Electronic-Police-Report-2016/4gc2-25he/data" target="_blank">https&#58;//data.nola.gov/Public-Safety-and-Preparedness/Electronic-Police-Report-2016/4gc2-25he/data</a>
| New York      | <a href="https://data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i" target="_blank">https&#58;//data.cityofnewyork.us/Public-Safety/NYPD-Complaint-Data-Historic/qgea-i56i</a>
| Philadelphia  | <a href="https://www.opendataphilly.org/dataset/crime-incidents" target="_blank">https&#58;//www.opendataphilly.org/dataset/crime-incidents</a>

-sandbox
&copy; 2018 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>